In [1]:
from fastai.vision.all import *
from fastai.tabular.all import *
from sklearn.model_selection import train_test_split
import gc

import os
set_seed(42)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
'''
------------- TRAIN IMG SIZES -------------
(640, 480)      15247
(320, 240)       5983
(3264, 2448)     1388
(3952, 1872)     1142
(4160, 3120)      590
(2592, 1944)      423
(153, 204)        279
(4128, 1952)      111
(2576, 1932)      111
(4608, 3456)      111
(3840, 2160)      106
(3456, 3456)      104
(256, 144)         88
(204, 153)         72
(3968, 2976)       69
(4160, 1872)       39
(240, 320)         29
(2560, 1920)       27
(260, 195)         27
(195, 260)         24
(3264, 1836)       16
(194, 194)         16
(4000, 1800)       12
(186, 248)          8
(204, 102)          8
(4128, 3096)        6
(4000, 3000)        4
(160, 120)          4
(122, 162)          3
(120, 160)          3
(162, 121)          3
(192, 144)          2
(3120, 4160)        2
(117, 260)          2
(248, 186)          2
(256, 192)          2
(150, 200)          1
(121, 161)          1
(216, 216)          1
(1600, 1200)        1
(120, 120)          1


------------- TEST IMG SIZES -------------
(640, 480)      5126
(320, 240)      1938
(3264, 2448)     440
(3952, 1872)     389
(4160, 3120)     192
(2592, 1944)     142
(153, 204)       102
(2576, 1932)      46
(4128, 1952)      41
(3456, 3456)      40
(3840, 2160)      31
(4608, 3456)      31
(3968, 2976)      27
(256, 144)        20
(204, 153)        19
(240, 320)        13
(4160, 1872)      12
(260, 195)         8
(2560, 1920)       8
(3264, 1836)       6
(194, 194)         5
(195, 260)         4
(186, 248)         3
(160, 120)         3
(204, 102)         2
(121, 162)         2
(120, 160)         2
(102, 204)         1
(120, 120)         1
(4000, 1800)       1
(4128, 3096)       1
(117, 260)         1
(122, 162)         1
(4000, 3000)       1
(256, 192)         1
(216, 216)         1
(1600, 1200)       1
(150, 200)         1
'''

'\n------------- TRAIN IMG SIZES -------------\n(640, 480)      15247\n(320, 240)       5983\n(3264, 2448)     1388\n(3952, 1872)     1142\n(4160, 3120)      590\n(2592, 1944)      423\n(153, 204)        279\n(4128, 1952)      111\n(2576, 1932)      111\n(4608, 3456)      111\n(3840, 2160)      106\n(3456, 3456)      104\n(256, 144)         88\n(204, 153)         72\n(3968, 2976)       69\n(4160, 1872)       39\n(240, 320)         29\n(2560, 1920)       27\n(260, 195)         27\n(195, 260)         24\n(3264, 1836)       16\n(194, 194)         16\n(4000, 1800)       12\n(186, 248)          8\n(204, 102)          8\n(4128, 3096)        6\n(4000, 3000)        4\n(160, 120)          4\n(122, 162)          3\n(120, 160)          3\n(162, 121)          3\n(192, 144)          2\n(3120, 4160)        2\n(117, 260)          2\n(248, 186)          2\n(256, 192)          2\n(150, 200)          1\n(121, 161)          1\n(216, 216)          1\n(1600, 1200)        1\n(120, 120)          1\n\n\n-----

In [3]:
df       = pd.read_csv('/kaggle/input/cgiar-eyes-on-the-ground-challenge/Train.csv')
test_df  = pd.read_csv('/kaggle/input/cgiar-eyes-on-the-ground-challenge/Test.csv')
df

,ID,filename,growth_stage,damage,extent,season
0,ID_1S8OOWQYCB,L427F01330C01S03961Rp02052.jpg,S,WD,0,SR2020
1,ID_0MD959MIZ0,L1083F00930C39S12674Ip.jpg,V,G,0,SR2021
2,ID_JRJCI4Q11V,24_initial_1_1463_1463.JPG,V,G,0,LR2020
3,ID_DBO3ZGI1GM,L341F00167C01S00324Rp14178.jpg,M,DR,60,SR2020
4,ID_ORZLWTEUUS,L1084F02394C39S13931Ip.jpg,V,G,0,SR2021
...,...,...,...,...,...,...
26063,ID_3II1SXC0ZO,L1084F03259C39S12149Rp41671.jpg,M,DR,30,SR2021
26064,ID_OE7OU9ZF4U,L406F04369C01S07190Rp22847.jpg,V,G,0,LR2021
26065,ID_20M531UIZZ,L134F00766C01S09784Rp26034.jpg,M,G,0,LR2021
26066,ID_BZBV2FH0KL,L1153F02464C01S00194Rp01561.jpg,F,G,0,SR2020


In [4]:
stratify_by = df['season']
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=stratify_by, random_state=42)

In [5]:
train_df[train_df['extent']!=0].shape[0], train_df[train_df['extent']==0].shape[0], valid_df[valid_df['extent']!=0].shape[0], valid_df[valid_df['extent']==0].shape[0]

(3622, 17232, 888, 4326)

In [6]:
train_df = pd.concat([train_df, train_df[train_df['extent']!=0]])

train_df[train_df['extent']!=0].shape[0], train_df[train_df['extent']==0].shape[0], valid_df[valid_df['extent']!=0].shape[0], valid_df[valid_df['extent']==0].shape[0]

(7244, 17232, 888, 4326)

In [7]:
train_df = pd.concat([train_df, train_df[train_df['extent']!=0]])

train_df[train_df['extent']!=0].shape[0], train_df[train_df['extent']==0].shape[0], valid_df[valid_df['extent']!=0].shape[0], valid_df[valid_df['extent']==0].shape[0]

(14488, 17232, 888, 4326)

In [8]:
import time
from fastai.callback.core import Callback

class TimeLimitCallback(Callback):
    def __init__(self, time_limit_hours):
        self.time_limit = time_limit_hours * 3600  # Convert hours to seconds
        self.start_time = time.time()

    def before_epoch(self):
        elapsed_time = time.time() - self.start_time
        if elapsed_time > self.time_limit:
            print(f"Training exceeded time limit of {self.time_limit / 3600} hours. Stopping training.")
            raise TimeLimitExceededException("Training exceeded time limit.")
            self.learn.stop = True  # Stop the training 
            
            
class TimeLimitExceededException(Exception):
    pass

def train(arch, size, item=Resize(480), finetune=True, epochs=15):
    time_limit_cb = TimeLimitCallback(time_limit_hours=7.5)
    dblock = DataBlock(blocks=(ImageBlock, RegressionBlock(n_out=1)),
               get_x=ColReader('filename', pref='/kaggle/input/cgiar-eyes-on-the-ground-challenge/content/content/train/'),
               get_y=ColReader('extent'),
               #splitter=RandomSplitter(valid_pct=0.1, seed=42),
               item_tfms=item,
               batch_tfms=aug_transforms(size=size, min_scale=0.75))

    # Create the data loaders
    dls = dblock.dataloaders(df)

    try:
        learn = vision_learner(dls, arch, metrics=rmse, cbs = [time_limit_cb]).to_fp16()
        if finetune:
            learn.fine_tune(epochs, 0.01)
            return learn.tta(dl=learn.dls.test_dl('/kaggle/input/cgiar-eyes-on-the-ground-challenge/content/content/test/' + test_df['filename']))
        else:
            learn.unfreeze()
            learn.fit_one_cycle(epochs, 0.01)
            return learn.tta(dl=learn.dls.test_dl('/kaggle/input/cgiar-eyes-on-the-ground-challenge/content/content/test/' + test_df['filename']))
    except TimeLimitExceededException:
        pass  # Handle the exception or perform any necessary cleanup here
    

In [9]:
res = 640,480

In [10]:
models = {
    'convnext_small_in22k': {
        (Resize(320,240), (224,224)),
    }, 'vit_small_patch16_224': {
        (Resize(320), 224),
     #   (Resize(480,320), 224),
    }
}

In [11]:
%%time
tta_res = []

for arch,details in models.items():
    for item,size in details:
        print('---',arch)
        print(size)
        print(item.name)
        tta_res.append(train(arch, size, item=item)) #, epochs=1))
        gc.collect()
        torch.cuda.empty_cache()

--- convnext_small_in22k
(224, 224)
Resize -- {'size': (320, 320), 'method': 240, 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name convnext_small_in22k to current convnext_small.fb_in22k.
  model = create_fn(


epoch,train_loss,valid_loss,_rmse,time
0,169.520020,157.191269,12.537594,12:54


epoch,train_loss,valid_loss,_rmse,time
0,146.635864,140.081863,11.835618,14:04
1,140.164001,129.603546,11.384356,14:06
2,127.111168,123.518463,11.113885,14:38
3,133.356476,133.203201,11.541369,14:48
4,125.021111,140.108597,11.836748,14:08
5,97.158394,121.252419,11.011469,14:03
6,90.998459,124.921036,11.176808,14:06
7,72.828285,125.097572,11.184703,14:05
8,68.170876,125.830414,11.217413,14:05
9,55.899326,126.326477,11.239505,14:07


--- vit_small_patch16_224
224
Resize -- {'size': (320, 320), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


epoch,train_loss,valid_loss,_rmse,time
0,164.793777,156.343140,12.503726,05:59


epoch,train_loss,valid_loss,_rmse,time
0,147.677795,130.398361,11.419209,06:07
1,131.006897,134.313232,11.589356,06:06
2,134.925339,129.534119,11.381305,06:06
3,136.751312,141.018341,11.875113,06:07
4,118.579239,127.218964,11.279139,06:05
5,117.963226,127.374542,11.286033,06:06
6,101.462334,130.403397,11.419431,06:09
7,92.895943,127.086586,11.273270,06:07
8,81.299644,120.365784,10.971134,06:03
9,69.925095,113.802032,10.667803,06:05


CPU times: user 3h 8min 58s, sys: 1h 6min 9s, total: 4h 15min 7s
Wall time: 5h 42min 49s


In [12]:
save_pickle('tta_res.pkl', tta_res)

In [13]:
tta_prs = first(zip(*tta_res))

In [14]:
avg_pr = torch.stack(tta_prs).mean(0)
avg_pr.shape

torch.Size([8663, 1])

In [15]:
preds    = torch.relu(avg_pr)

In [16]:
submission_df = pd.DataFrame({
    'ID': test_df['ID'],
    'extent': preds.squeeze().numpy()  # Use .squeeze() to remove any extra dimensions from the predictions
})

In [17]:
submission_df.to_csv('SampleSubmission.csv', index=False)